In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'
file_names = ['Serie_A']
years = range(2017, 2018)

# processor = FootballDataProcessor('../Data/Serie_A_2014_games.json')
# final_df = processor.process(data_folder, file_names, years)
# final_df.rename(columns={'player_id': 'player'}, inplace=True)


# # Garantir que ambas as colunas 'player' estejam no tipo string
# final_df['player'] = final_df['player'].astype('string')
# id_players['player'] = id_players['player'].astype('string')

# # Realizar o merge sem utilizar índices, mas mantendo a coluna 'player'
# final_df_ = pd.merge(final_df, id_players, on='player', how='left')

# # Definir os nomes dos times que deseja analisar
# team_1_name = 'Criciúma / SC'  # Substitua pelo nome real do time 1 
# team_2_name = 'Palmeiras / SP'  # Substitua pelo nome real do time 2

# # Criar uma instância de FootballMatchRating
# rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

# # Atualizar as ratings
# updated_match_data = rating_calculator.update_ratings()

# # Garantir que 'player' em updated_match_data está como string
# updated_match_data['player'] = updated_match_data['player'].astype('string')

# # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
# updated_values = id_players[['player']].merge(
#     updated_match_data[['player', 'rating', 'games_played']],
#     on='player',
#     how='left'
# )

# # Atualizar apenas as linhas correspondentes em id_players
# id_players.update(updated_values)

In [2]:
df_jogos_2014 = pd.read_json('../Data/Serie_A_2017_games.json')
df_jogos_2014_ = df_jogos_2014.T

for index, row in df_jogos_2014_.iterrows():
    if index == 2:

        processor = FootballDataProcessor(df_jogos_2014_, index-1)
        final_df = processor.process(data_folder, file_names, years)
        final_df.rename(columns={'player_id': 'player'}, inplace=True)

        final_df['player'] = final_df['player'].astype('string')
        id_players['player'] = id_players['player'].astype('string')

        final_df_ = pd.merge(final_df, id_players, on='player', how='left')

        team_1_name = df_jogos_2014_.iloc[index-1,0]
        team_2_name = df_jogos_2014_.iloc[index-1,1]


        # Criar uma instância de FootballMatchRating
        rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

        # Atualizar as ratings
        updated_match_data = rating_calculator.update_ratings()
        # Garantir que 'player' em updated_match_data está como string
        updated_match_data['player'] = updated_match_data['player'].astype('string')
        # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
        updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
        updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

        updated_values = id_players[['player']].merge(
            updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
            on='player',
            how='left'
        )

        # Atualizar apenas as linhas corespondentes em id_players
        id_players.update(updated_values)

id_players


                                               player                   team  \
0   38Alex Muralha Alex Roberto Santana ... T(g)P1...          Flamengo / RJ   
1     5Willian Arão Willian Souza Arao d ... TP293420          Flamengo / RJ   
2    8Marcio Ara ... Marcio Rodrigues Araujo TP162907          Flamengo / RJ   
3         9Guerrero Jose Paolo Guerrero  ... TP420184          Flamengo / RJ   
4          13Trauco Miguel Angel Trauco  ... TP565356          Flamengo / RJ   
5           15Rever Rever Humberto Alves ... TP167509          Flamengo / RJ   
6            21Pará Marcos Rogerio Ricci ... TP165065          Flamengo / RJ   
7            22Everton Everton Cardoso Silva TP178361          Flamengo / RJ   
8          28Berrio Orlando Enrique Berr ... TP565602          Flamengo / RJ   
9         33Rafael Vaz Rafael Vaz dos Santos TP171406          Flamengo / RJ   
10  42Matheus Sa ... Matheus Gonçalves Savio TP395014          Flamengo / RJ   
11         2Rodinei Rodinei Marcelo de A

,player,rating,age,games_played,player_name,time_jogador
0,135687,1500.0000,25,50,NaN,NaN
1,380762,1500.0000,25,50,NaN,NaN
2,172028,1500.0000,25,50,NaN,NaN
3,439179,1500.0000,25,50,NaN,NaN
4,178827,1500.0000,25,50,NaN,NaN
...,...,...,...,...,...,...
717,364085,1500.0000,25,50,NaN,NaN
718,137990,1500.0000,25,50,NaN,NaN
719,165065,1497.1987,25,51,21Pará Marcos Rogerio Ricci ...,Flamengo / RJ
720,178620,1500.0000,25,50,NaN,NaN


In [3]:
x = id_players.sort_values(by='rating')
x.to_excel('teste.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
152,395014,1491.323563,25,51,42Matheus Sa ... Matheus Gonçalves Savio,Flamengo / RJ
417,540839,1494.546272,25,51,10Cazares Juan Ramon Cazares S...,Atlético Mineiro / MG
719,165065,1497.198700,25,51,21Pará Marcos Rogerio Ricci ...,Flamengo / RJ
593,162907,1497.198700,25,51,8Marcio Ara ... Marcio Rodrigues Araujo,Flamengo / RJ
537,167509,1497.198700,25,51,15Rever Rever Humberto Alves ...,Flamengo / RJ
...,...,...,...,...,...,...
346,181262,1502.728362,25,51,5Rafael Car ... Rafael de Souza Pereira,Atlético Mineiro / MG
702,162248,1502.728362,25,51,6Fabio Santos Fabio Santos Romeu,Atlético Mineiro / MG
70,318309,1502.728362,25,51,30Gabriel Co ... Gabriel Costa Franca,Atlético Mineiro / MG
454,169374,1502.728362,25,51,26Felipe Felipe Augusto Santana,Atlético Mineiro / MG
